In [ ]:
!pip install --quiet google-cloud-bigquery pandas requests
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd, requests
from datetime import datetime


In [ ]:
project_id = "worldbank-data-474912"
raw_dataset = "crypto_raw"
clean_dataset = "crypto_clean"
table_name = "coingecko_data"
bq_client = bigquery.Client(project=project_id)


for ds in [raw_dataset, clean_dataset]:
    bq_client.create_dataset(ds, exists_ok=True)
    print(f"Dataset ready: {project_id}.{ds}")


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time

def fetch_coingecko_data(pages=5, per_page=250, delay=2):
    all_data = []
    for page in range(1, pages + 1):
        url = "https://api.coingecko.com/api/v3/coins/markets"
        params = {
            "vs_currency": "usd",
            "order": "market_cap_desc",
            "per_page": per_page,
            "page": page,
            "sparkline": False
        }
        resp = requests.get(url, params=params, timeout=30)
        resp.raise_for_status()
        print(f"Fetched page {page} with {len(resp.json())} rows")
        all_data.extend(resp.json())


        time.sleep(delay)

    df = pd.DataFrame(all_data)
    df["fetch_time"] = datetime.utcnow().isoformat()
    print(f"Total rows fetched: {len(df)}")
    return df

df_raw = fetch_coingecko_data(pages=5, delay=2)
df_raw.shape


In [ ]:
table_id = f"{project_id}.{raw_dataset}.{table_name}"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
)

job = bq_client.load_table_from_dataframe(df_raw, table_id, job_config=job_config)
job.result()

print(f"Loaded {len(df_raw)} rows into {table_id}")


In [ ]:
clean_table_id = f"{project_id}.{clean_dataset}.{table_name}"

query = f"""
CREATE OR REPLACE TABLE `{clean_table_id}` AS
SELECT
  id,
  symbol,
  name,
  current_price,
  market_cap,
  total_volume,
  high_24h,
  low_24h,
  price_change_percentage_24h,
  last_updated,
  fetch_time
FROM `{table_id}`
WHERE current_price IS NOT NULL
"""
bq_client.query(query).result()
print(f"Created clean table: {clean_table_id}")


In [ ]:
query = f"SELECT COUNT(*) AS total_rows FROM `{clean_table_id}`"
bq_client.query(query).to_dataframe()


In [ ]:
from google.cloud import storage

project_id = "worldbank-data-474912"

storage_client = storage.Client(project=project_id)


bucket_name = "worldbank-data-bucket-12345"

bucket = storage_client.create_bucket(bucket_name)
print(f"Bucket {bucket_name} created successfully in project {project_id}!")


In [ ]:
from google.cloud import storage

bucket = storage_client.get_bucket(bucket_name)

local_file = "worldbank_data.csv"
blob = bucket.blob("worldbank_data.csv")


blob.upload_from_filename(local_file)

print(f"File {local_file} uploaded to bucket {bucket_name} successfully!")


In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=project_id)

dataset_id = "worldbank_dataset"
dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")

dataset = bq_client.create_dataset(dataset_ref, exists_ok=True)
print(f"Dataset {dataset_id} created successfully!")


In [ ]:

table_id = f"{project_id}.{dataset_id}.gdp_data"

file_path = "worldbank_data.csv"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)


with open(file_path, "rb") as source_file:
    load_job = bq_client.load_table_from_file(
        source_file,
        table_id,
        job_config=job_config
    )

load_job.result()
print(f"Table {table_id} loaded successfully!")
